# AB - Module 5

Однажды мы с вами сравнивали метрики между двумя моделями на одной и той же выборке. 

Чаще всего метрики по объектам представляют собой ненормальное распределение, поэтому лучше оценивать их различия непараметрическими статистическими критериями.

Давайте попрактикумемся в этом. Возьмём уже привычный нам датасет со средними ценами на дома.

Здесь мы ещё сделаем нормализацию фичей и зафиксируем разбиение.

Далее обучите Ridge-регрессии из библиотеки sklearn с параметрами alpha=1 и alpha=10. Предсказания сделайте на всей выборке по фолдам с помощью функции cross_val_predict (по очереди каждый фолд будет становиться тестовым), не забудьте передать туда наше разбиение.

Посчитайте MSE по каждому объекту выборки и сравните равенство их медиан с помощью критерия знаков. Сам критерий знаков реализуйте вручную, его нулевое распределение представляет собой биномиальное распределение, с котором мы уже работали ранее.

Какой вывод вы можете сделать о различии этих моделей? Альтернатива двусторонняя, уровень значимости 0.05.

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

data = fetch_california_housing(as_frame=True, return_X_y=True)
df, target = data[0], data[1] 
df = StandardScaler().fit_transform(df)

cv = KFold(n_splits=4, shuffle=True, random_state=6)

In [11]:
from sklearn.linear_model import Ridge

ridge_1 = Ridge(alpha=10)
ridge_2 = Ridge(alpha=1)

mses_1 = cross_val_score(ridge_1, df, target, cv=cv, scoring='neg_mean_squared_error')
mses_2 = cross_val_score(ridge_2, df, target, cv=cv, scoring='neg_mean_squared_error')


In [13]:
import numpy as np

mses_1 = np.sort(np.abs(mses_1))
mses_2 = np.sort(np.abs(mses_2))

In [14]:
mses_1 > mses_2

array([ True,  True,  True, False])

In [20]:
from scipy.stats import binomtest

binomtest(3, 4, 0.5) 


BinomTestResult(k=3, n=4, alternative='two-sided', statistic=0.75, pvalue=0.625)

В прошлом уроке мы сравнивали средние доходы (MedInc) между районами, где дома моложе 30 лет (включительно) и старше 30 лет (HouseAge). Однако тогда мы заметили, что случайная величина распределена не нормально.

Давайте воспользуемся непараметрическими статистическими критериями, которые не делают предположений на вид распределения.

Примените критерий Манна-Уитни-Уилкоксона к выборке и сравните, различаются ли распределения между выборками. Какой вывод вы можете сделать? Альтернатива двусторонняя.

In [25]:
from scipy.stats import mannwhitneyu

data = fetch_california_housing(as_frame=True, return_X_y=True)
df, target = data[0], data[1]

first_sample = df[df['HouseAge'] > 30]['MedInc']
second_sample = df[df['HouseAge'] < 30]['MedInc']

mannwhitneyu(x=first_sample, y=second_sample, method='asymptotic')


MannwhitneyuResult(statistic=44225886.0, pvalue=1.1132480655784223e-54)